In [39]:
import os
import pandas as pd
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

def load_data(file):
    return pd.read_csv(file)

path = "datas/"
general_data = load_data(path +"general_data.csv")
employee_survey_data = load_data(path +"employee_survey_data.csv")
manager_survey_data = load_data(path +"manager_survey_data.csv")
in_time = load_data(path +"in_time.csv")
out_time = load_data(path +"out_time.csv")
#general_data.info()
#test= general_data.drop(columns=['Attrition','BusinessTravel'],axis=1).select_dtypes(include=["object"]).keys()
general_data
#print(test)

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,...,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,...,1.0,Y,11,8,0,1.0,6,1,0,0
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,...,0.0,Y,23,8,1,6.0,3,5,1,4
2,32,No,Travel_Frequently,Research & Development,17,4,Other,1,3,Male,...,1.0,Y,15,8,3,5.0,2,5,0,3
3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,1,4,Male,...,3.0,Y,11,8,3,13.0,5,8,7,5
4,32,No,Travel_Rarely,Research & Development,10,1,Medical,1,5,Male,...,4.0,Y,12,8,2,9.0,2,6,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,42,No,Travel_Rarely,Research & Development,5,4,Medical,1,4406,Female,...,3.0,Y,17,8,1,10.0,5,3,0,2
4406,29,No,Travel_Rarely,Research & Development,2,4,Medical,1,4407,Male,...,2.0,Y,15,8,0,10.0,2,3,0,2
4407,25,No,Travel_Rarely,Research & Development,25,2,Life Sciences,1,4408,Male,...,0.0,Y,20,8,0,5.0,4,4,1,2
4408,42,No,Travel_Rarely,Sales,18,2,Medical,1,4409,Male,...,0.0,Y,14,8,1,10.0,2,9,7,8


In [40]:
def to_datetime(strdate):
    if type(strdate) == str:
        return datetime.strptime(strdate, "%Y-%m-%d %H:%M:%S").timestamp()
    else:
        return 0

def get_working_time(in_time_data, out_time_data):
    out_time_df = (
        out_time_data
        .iloc[:,1:]
        .applymap(lambda x: to_datetime(x))
    )
    in_time_df = (
        in_time_data
        .iloc[:,1:]
        .applymap(lambda x: to_datetime(x))
    )
    hours_per_day = out_time_df - in_time_df
    in_time_data["MeanWorkingHours"] = hours_per_day.mean(axis = 1)/3600
    return in_time_data[["EmployeeID", "MeanWorkingHours"]]

In [41]:
class MergeDataFrame(BaseEstimator, TransformerMixin):
    def __init__(self, employee_survey_data, manager_survey_data, in_time, out_time):
        self.employee_survey_data = employee_survey_data
        self.manager_survey_data = manager_survey_data
        self.in_time= in_time
        self.out_time = out_time
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X = X.join(self.employee_survey_data.set_index("EmployeeID"), on="EmployeeID")
        X = X.join(self.manager_survey_data.set_index("EmployeeID"), on="EmployeeID")
        X = pd.merge(X, get_working_time(in_time, out_time), on='EmployeeID', how='inner')

        return X

In [42]:
class DeleteUnEthicColumn(BaseEstimator, TransformerMixin):
    def __init__(self, array):
        self.array = array

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        for i in self.array:
            X.drop(i, axis=1, inplace=True)
        return X


In [43]:
class OrgDataFrame(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X["Attrition"] = X["Attrition"].replace(to_replace=['No', 'Yes'], value=[-1, 1])
        X["BusinessTravel"] = X["BusinessTravel"].replace(to_replace=['Non-Travel', 'Travel_Rarely', 'Travel_Frequently'], value=[1, 2, 3])
        return X


In [44]:
'''class ScaleData(BaseEstimator, TransformerMixin):
    def __init__(self, array):
        self.array = array

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        std_scale = StandardScaler().fit()
        X_scaled = pd.DataFrame(std_scale.transform(X), columns=X.columns)
        return X_scaled'''

'class ScaleData(BaseEstimator, TransformerMixin):\n    def __init__(self, array):\n        self.array = array\n\n    def fit(self, X, y=None):\n        return self\n\n    def transform(self, X, y=None):\n        std_scale = StandardScaler().fit()\n        X_scaled = pd.DataFrame(std_scale.transform(X), columns=X.columns)\n        return X_scaled'

In [45]:
def build_pipeline(X):

#Pipeline one for preparation
    #Merge all files in one and change in and out datetime
    prepa_pipeline= Pipeline([
        ("merge", MergeDataFrame(load_data(path +"employee_survey_data.csv"),
                                 load_data(path +"manager_survey_data.csv"),
                                 load_data(path +"in_time.csv"),
                                 load_data(path +"out_time.csv"))),
        #delete useless columns and unEthic
        ("delete", DeleteUnEthicColumn([
                                        "Age",
                                        "Over18",
                                        "Gender",
                                        "MaritalStatus",
                                        "EmployeeCount",
                                        "StandardHours",
                                        "EmployeeID"
        
        ])),
        #encode Attrition and BusinessTravel columns
        ("label",OrgDataFrame()),
    ])
    #create a DataFrame with pipeline 
    data_pipeline = prepa_pipeline.fit_transform(X)
    ###print(data_pipeline)
    #create label for classification
    y = data_pipeline['Attrition']
    
    #list all columns who are object to onehot encode them
    object_attribs= data_pipeline.drop(columns=['Attrition','BusinessTravel'],axis=1).select_dtypes(include=["object"]).keys()
    ###print(type(object_attribs))
    ###num_search = data_pipeline.drop(columns=['Attrition'],axis=1).select_dtypes(include=[np.number])
    #list all columns who are number to scale them
    num_search = data_pipeline.select_dtypes(include=[np.number])
    num_attribs = list(num_search)
    #print(object_attribs)
    print(num_attribs)

    #create onehot encode step pipeline   
    onehot_pipeline = Pipeline(steps=[
        ('onehot',OneHotEncoder())
    ])
    #create scale step pipeline
    scale_pipeline = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])
    #create column transformers 
    pipeline = ColumnTransformer(transformers=[
        ("numerisation",scale_pipeline,num_attribs),
        ("one", onehot_pipeline, object_attribs) 
        ],
        remainder='drop',
        n_jobs=-1)

    #create final pipeline
    full_pipeline = Pipeline(steps=[
        ("pipeline", pipeline),
    ])

    #print pipeline
    from sklearn import set_config
    set_config(display='diagram')
    display(full_pipeline)
    
    #execute pipeline
    data_fullpipeline = full_pipeline.fit_transform(data_pipeline)
    


    #get all columns name to recreate index on dataFrame
    cols_names=full_pipeline.named_steps["pipeline"].named_transformers_["one"].named_steps["onehot"].get_feature_names_out(object_attribs)
    all_cols=np.concatenate([num_attribs,cols_names])
    data_fullpipeline=pd.DataFrame(data_fullpipeline,columns=all_cols)
    
    #create unlabeled dataset
    X = data_fullpipeline.drop(columns=['Attrition'],axis=1)

    
    return data_fullpipeline, X, y

In [46]:
data, X, y = build_pipeline(general_data)

['Attrition', 'BusinessTravel', 'DistanceFromHome', 'Education', 'JobLevel', 'MonthlyIncome', 'NumCompaniesWorked', 'PercentSalaryHike', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager', 'EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance', 'JobInvolvement', 'PerformanceRating', 'MeanWorkingHours']


Pipeline(steps=[('pipeline',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('numerisation',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('std_scaler',
                                                                   StandardScaler())]),
                                                  ['Attrition',
                                                   'BusinessTravel',
                                                   'DistanceFromHome',
                                                   'Education', 'JobLevel',
                                                   'MonthlyIncome',
                                                   'NumCompaniesWorked',
                                                   'PercentSalaryHike',
                                                   'StockOptionLevel',
                                                   'TotalWorkingYears',
                                                   'TrainingTimesLastYear',
                                                   'YearsAtCompany',
                                                   'YearsSinceLastPromotion',
                                                   'YearsWithCurrManager',
                                                   'EnvironmentSatisfaction',
                                                   'JobSatisfaction',
                                                   'WorkLifeBalance',
                                                   'JobInvolvement',
                                                   'PerformanceRating',
                                                   'MeanWorkingHours']),
                                                 ('one',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder())]),
                                                  Index(['Department', 'EducationField', 'JobRole'], dtype='object'))]))])

In [47]:
data

,Attrition,BusinessTravel,DistanceFromHome,Education,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,...,EducationField_Technical Degree,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative
0,-0.438422,-0.162399,-0.393938,-0.891688,-0.961486,1.405136,-0.678464,-1.150554,-0.932014,-1.322079,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.280906,1.717339,0.099639,-1.868426,-0.961486,-0.491661,-1.079486,2.129306,0.241988,-0.678877,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,-0.438422,1.717339,0.963398,1.061787,1.749610,2.725053,-0.678464,-0.057267,2.589994,-0.807517,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-0.438422,-2.042138,-0.887515,2.038524,0.845911,0.386301,0.123580,-1.150554,2.589994,0.221606,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.438422,-0.162399,0.099639,-1.868426,-0.961486,-0.884109,0.524602,-0.877232,1.415991,-0.292956,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,-0.438422,-0.162399,-0.517332,1.061787,-0.961486,-0.100700,0.123580,0.489376,0.241988,-0.164315,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4406,-0.438422,-0.162399,-0.887515,1.061787,-0.961486,-0.812504,-0.277442,-0.057267,-0.932014,-0.164315,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4407,-0.438422,-0.162399,1.950552,-0.891688,-0.057788,-0.595138,-1.079486,1.309341,-0.932014,-0.807517,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4408,-0.438422,-0.162399,1.086793,-0.891688,-0.961486,-0.872210,-1.079486,-0.330589,0.241988,-0.164315,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
X    

,BusinessTravel,DistanceFromHome,Education,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,...,EducationField_Technical Degree,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative
0,-0.162399,-0.393938,-0.891688,-0.961486,1.405136,-0.678464,-1.150554,-0.932014,-1.322079,2.483396,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.717339,0.099639,-1.868426,-0.961486,-0.491661,-1.079486,2.129306,0.241988,-0.678877,0.155707,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.717339,0.963398,1.061787,1.749610,2.725053,-0.678464,-0.057267,2.589994,-0.807517,-0.620189,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-2.042138,-0.887515,2.038524,0.845911,0.386301,0.123580,-1.150554,2.589994,0.221606,1.707500,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.162399,0.099639,-1.868426,-0.961486,-0.884109,0.524602,-0.877232,1.415991,-0.292956,-0.620189,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,-0.162399,-0.517332,1.061787,-0.961486,-0.100700,0.123580,0.489376,0.241988,-0.164315,1.707500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4406,-0.162399,-0.887515,1.061787,-0.961486,-0.812504,-0.277442,-0.057267,-0.932014,-0.164315,-0.620189,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4407,-0.162399,1.950552,-0.891688,-0.057788,-0.595138,-1.079486,1.309341,-0.932014,-0.807517,0.931603,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4408,-0.162399,1.086793,-0.891688,-0.961486,-0.872210,-1.079486,-0.330589,0.241988,-0.164315,-0.620189,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
y

0      -1
1       1
2      -1
3      -1
4      -1
       ..
4405   -1
4406   -1
4407   -1
4408   -1
4409   -1
Name: Attrition, Length: 4410, dtype: int64